In [1]:
import pandas as pd
# !pip install xlrd
import requests
from bs4 import BeautifulSoup
import time
import re
from collections import OrderedDict


# Initial Data Collection

In [2]:
# HUD provides two data sets: Point-in-Time (PIT) estimates, and Housing Inventory Count (HIC, availability of assistance housing e.g. shelters). 
# I chose to use PIT because I'm working with *total* number of homeless individuals, so 
# assistance housing would apply to a subset of the relevant data.

df_pit_2018 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2018', usecols="A:D").dropna() 
# from the .xlsx file, I know that '2018' is the only year/sheet with "CoC Category" column.
df_pit_2017 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2017', usecols="A:C").dropna()
df_pit_2016 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2016', usecols="A:C").dropna()
df_pit_2015 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2015', usecols="A:C").dropna()
df_pit_2014 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2014', usecols="A:C").dropna()
df_pit_2013 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2013', usecols="A:C").dropna()
df_pit_2012 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2012', usecols="A:C").dropna()
df_pit_2011 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2011', usecols="A:C").dropna()
df_pit_2010 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2010', usecols="A:C").dropna()
df_pit_2009 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2009', usecols="A:C").dropna()
df_pit_2008 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2008', usecols="A:C").dropna()
df_pit_2007 = pd.read_excel('2007-2018-PIT-Counts-by-CoC.xlsx', sheet_name='2007', usecols="A:C").dropna()

# cell executes in about 30 seconds.

In [3]:
df_pit_2018

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018"
0,AK-500,Anchorage CoC,Other Urban CoCs,1094
1,AK-501,Alaska Balance of State CoC,Rural CoCs,922
2,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",Suburban CoCs,901
3,AL-501,Mobile City & County/Baldwin County CoC,Other Urban CoCs,551
4,AL-502,Florence/Northwest Alabama CoC,Rural CoCs,256
...,...,...,...,...
393,WV-500,"Wheeling, Weirton Area CoC",Rural CoCs,108
394,WV-501,"Huntington/Cabell, Wayne Counties CoC",Rural CoCs,190
395,WV-503,"Charleston/Kanawha, Putnam, Boone, Clay Counti...",Suburban CoCs,317
396,WV-508,West Virginia Balance of State CoC,Rural CoCs,628


In [4]:
# compare sizes of two dfs
df_pit_2018['CoC Number'].size,df_pit_2017['CoC Number'].size

(398, 397)

In [5]:
df_pit_2017.nunique()

CoC Number                397
CoC Name                  397
Overall Homeless, 2017    358
dtype: int64

In [6]:
# look at column names for both dfs to determine where I want to merge
df_pit_2018.columns, df_pit_2017.columns

(Index(['CoC Number', 'CoC Name', 'CoC Category', 'Overall Homeless, 2018'], dtype='object'),
 Index(['CoC Number', 'CoC Name', 'Overall Homeless, 2017'], dtype='object'))

In [7]:
# check that relevant column names are the same for each df
df_pit_2018.columns.values[1] == df_pit_2017.columns.values[1]

True

In [8]:
# try merging two columns (data years) into a new dataframe
df_pit_all = pd.merge(df_pit_2018, df_pit_2017[['CoC Number','Overall Homeless, 2017']], on='CoC Number', how='outer')
df_pit_all

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018","Overall Homeless, 2017"
0,AK-500,Anchorage CoC,Other Urban CoCs,1094,1128
1,AK-501,Alaska Balance of State CoC,Rural CoCs,922,717
2,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",Suburban CoCs,901,1092
3,AL-501,Mobile City & County/Baldwin County CoC,Other Urban CoCs,551,606
4,AL-502,Florence/Northwest Alabama CoC,Rural CoCs,256,155
...,...,...,...,...,...
393,WV-500,"Wheeling, Weirton Area CoC",Rural CoCs,108,118
394,WV-501,"Huntington/Cabell, Wayne Counties CoC",Rural CoCs,190,205
395,WV-503,"Charleston/Kanawha, Putnam, Boone, Clay Counti...",Suburban CoCs,317,319
396,WV-508,West Virginia Balance of State CoC,Rural CoCs,628,667


In [9]:
df_pit_all = pd.merge(df_pit_all, df_pit_2016[['CoC Number','Overall Homeless, 2016']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2015[['CoC Number','Overall Homeless, 2015']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2014[['CoC Number','Overall Homeless, 2014']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2013[['CoC Number','Overall Homeless, 2013']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2012[['CoC Number','Overall Homeless, 2012']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2011[['CoC Number','Overall Homeless, 2011']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2010[['CoC Number','Overall Homeless, 2010']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2009[['CoC Number','Overall Homeless, 2009']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2008[['CoC Number','Overall Homeless, 2008']], on='CoC Number', how='outer')
df_pit_all = pd.merge(df_pit_all, df_pit_2007[['CoC Number','Overall Homeless, 2007']], on='CoC Number', how='outer')
df_pit_all

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018","Overall Homeless, 2017","Overall Homeless, 2016","Overall Homeless, 2015","Overall Homeless, 2014","Overall Homeless, 2013","Overall Homeless, 2012","Overall Homeless, 2011","Overall Homeless, 2010","Overall Homeless, 2009","Overall Homeless, 2008","Overall Homeless, 2007"
0,AK-500,Anchorage CoC,Other Urban CoCs,1094,1128,1105,1208,1023,1122,1147,1223,1231.0,1267.0,1023.0,974
1,AK-501,Alaska Balance of State CoC,Rural CoCs,922,717,835,748,761,824,766,905,632.0,725.0,623.0,668
2,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",Suburban CoCs,901,1092,1228,1153,1329,1469,1707,1950,2273.0,2273.0,2104.0,2104
3,AL-501,Mobile City & County/Baldwin County CoC,Other Urban CoCs,551,606,623,578,598,493,634,718,883.0,747.0,524.0,649
4,AL-502,Florence/Northwest Alabama CoC,Rural CoCs,256,155,337,245,209,223,192,244,170.0,281.0,249.0,265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,MA-514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,636.0,747
405,AR-509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
406,AR-510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322
407,NY-515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.


# Exploratory Data Analysis and Data Cleaning

In [10]:
# work in a new df that is a copy of df_pit_all. !! STOP WORKING WITH df_pit_all !!
df_pit_clean = df_pit_all.copy()

In [11]:
# drop all rows (CoCs/cities) where there is data missing for any year in the range.
df_pit_clean.dropna(inplace=True)
df_pit_clean

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018","Overall Homeless, 2017","Overall Homeless, 2016","Overall Homeless, 2015","Overall Homeless, 2014","Overall Homeless, 2013","Overall Homeless, 2012","Overall Homeless, 2011","Overall Homeless, 2010","Overall Homeless, 2009","Overall Homeless, 2008","Overall Homeless, 2007"
0,AK-500,Anchorage CoC,Other Urban CoCs,1094,1128,1105,1208,1023,1122,1147,1223,1231.0,1267.0,1023.0,974
1,AK-501,Alaska Balance of State CoC,Rural CoCs,922,717,835,748,761,824,766,905,632.0,725.0,623.0,668
2,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",Suburban CoCs,901,1092,1228,1153,1329,1469,1707,1950,2273.0,2273.0,2104.0,2104
3,AL-501,Mobile City & County/Baldwin County CoC,Other Urban CoCs,551,606,623,578,598,493,634,718,883.0,747.0,524.0,649
4,AL-502,Florence/Northwest Alabama CoC,Rural CoCs,256,155,337,245,209,223,192,244,170.0,281.0,249.0,265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,WV-500,"Wheeling, Weirton Area CoC",Rural CoCs,108,118,91,114,97,89,116,124,197.0,87.0,92.0,118
394,WV-501,"Huntington/Cabell, Wayne Counties CoC",Rural CoCs,190,205,228,227,218,227,274,238,286.0,195.0,264.0,331
395,WV-503,"Charleston/Kanawha, Putnam, Boone, Clay Counti...",Suburban CoCs,317,319,320,385,360,411,416,394,406.0,382.0,363.0,325
396,WV-508,West Virginia Balance of State CoC,Rural CoCs,628,667,748,1109,1338,1513,1604,1455,1375.0,1003.0,1297.0,1635


In [12]:
df_pit_clean.info()
# some of the int values are stored as objects, so covert all "Overall Homeless" columns to ints.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384 entries, 0 to 397
Data columns (total 15 columns):
CoC Number                384 non-null object
CoC Name                  384 non-null object
CoC Category              384 non-null object
Overall Homeless, 2018    384 non-null object
Overall Homeless, 2017    384 non-null object
Overall Homeless, 2016    384 non-null object
Overall Homeless, 2015    384 non-null object
Overall Homeless, 2014    384 non-null object
Overall Homeless, 2013    384 non-null object
Overall Homeless, 2012    384 non-null object
Overall Homeless, 2011    384 non-null object
Overall Homeless, 2010    384 non-null float64
Overall Homeless, 2009    384 non-null float64
Overall Homeless, 2008    384 non-null float64
Overall Homeless, 2007    384 non-null object
dtypes: float64(3), object(12)
memory usage: 48.0+ KB


In [13]:
for i in range(2007,2019):
    df_pit_clean[f'Overall Homeless, {i}'] = pd.to_numeric(df_pit_clean[f'Overall Homeless, {i}'])
    print(df_pit_clean[f'Overall Homeless, {i}'].dtype)

int64
float64
float64
float64
int64
int64
int64
int64
int64
int64
int64
int64


In [14]:
df_pit_clean.describe()

,"Overall Homeless, 2018","Overall Homeless, 2017","Overall Homeless, 2016","Overall Homeless, 2015","Overall Homeless, 2014","Overall Homeless, 2013","Overall Homeless, 2012","Overall Homeless, 2011","Overall Homeless, 2010","Overall Homeless, 2009","Overall Homeless, 2008","Overall Homeless, 2007"
count,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000
mean,1428.585938,1424.742188,1423.440104,1461.669271,1492.523438,1527.304688,1608.281250,1612.656250,1650.059896,1632.856771,1657.421875,1675.007812
std,4901.510569,4876.719717,4523.891018,4536.333081,4085.640973,3960.242999,3682.332716,3567.985913,3630.432495,3493.616123,3980.026793,3968.412980
min,15.000000,10.000000,20.000000,9.000000,13.000000,7.000000,14.000000,22.000000,11.000000,11.000000,10.000000,28.000000
25%,277.750000,287.500000,309.500000,334.500000,340.500000,348.750000,357.250000,369.000000,368.500000,340.000000,326.250000,359.750000
50%,574.000000,574.000000,593.000000,635.500000,668.500000,710.000000,747.000000,744.000000,736.500000,747.500000,719.500000,710.000000
75%,1195.250000,1233.500000,1335.500000,1359.750000,1507.000000,1561.000000,1616.250000,1540.750000,1557.000000,1544.500000,1580.250000,1623.000000
max,78676.000000,76501.000000,73523.000000,75323.000000,67810.000000,64060.000000,56672.000000,51123.000000,53187.000000,49343.000000,50261.000000,50372.000000


In [15]:
df_pit_clean.reset_index(drop=True, inplace=True)

In [16]:
# make sure all CoC Numbers match the general format: 'XX-###'
coc_regex = re.compile(r"^[A-Z]{2}-\d{3}.") # any two letters plus - plus any three digits plus any character
                                            # to find any entries without this format
df_pit_clean["CoC Number"].apply(lambda x: bool(coc_regex.search(x))).value_counts()

False    383
True       1
Name: CoC Number, dtype: int64

In [17]:
# trying to display the row that doesn't saitsfy the regex test
df_pit_clean[df_pit_clean["CoC Number"].apply(lambda x: bool(coc_regex.search(x)))]

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018","Overall Homeless, 2017","Overall Homeless, 2016","Overall Homeless, 2015","Overall Homeless, 2014","Overall Homeless, 2013","Overall Homeless, 2012","Overall Homeless, 2011","Overall Homeless, 2010","Overall Homeless, 2009","Overall Homeless, 2008","Overall Homeless, 2007"
210,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",Major Cities,1798,1671,1699,1686,2193,2691,2705,2992,2153.0,1809.0,2260.0,1786


In [18]:
# do an inplace edit of that CoC Number value
df_pit_clean.iloc[210,0]=df_pit_clean.iloc[210,0][:-1]

In [19]:
df_pit_clean.iloc[210,0]

'MO-604'

In [20]:
df_pit_clean[df_pit_clean['CoC Number']=='OH-505']
# df_pit_clean['CoC Number']=='OH-505'

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018","Overall Homeless, 2017","Overall Homeless, 2016","Overall Homeless, 2015","Overall Homeless, 2014","Overall Homeless, 2013","Overall Homeless, 2012","Overall Homeless, 2011","Overall Homeless, 2010","Overall Homeless, 2009","Overall Homeless, 2008","Overall Homeless, 2007"
284,OH-505,"Dayton, Kettering/Montgomery County CoC",Suburban CoCs,680,619,751,971,791,1041,1081,986,884.0,837.0,844.0,785


In [21]:
maxima = df_pit_clean.describe().loc['max'] # takes the corresponding row of the "describe" matrix
# number = 78676
maxima

Overall Homeless, 2018    78676.0
Overall Homeless, 2017    76501.0
Overall Homeless, 2016    73523.0
Overall Homeless, 2015    75323.0
Overall Homeless, 2014    67810.0
Overall Homeless, 2013    64060.0
Overall Homeless, 2012    56672.0
Overall Homeless, 2011    51123.0
Overall Homeless, 2010    53187.0
Overall Homeless, 2009    49343.0
Overall Homeless, 2008    50261.0
Overall Homeless, 2007    50372.0
Name: max, dtype: float64

In [23]:
for i, value in maxima.items():
    print(df_pit_clean[df_pit_clean[i]==value]['CoC Name']);
    
# as expected, the CoC with the most homeless people every year is New York City.

271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object
271    New York City CoC
Name: CoC Name, dtype: object


In [24]:
# now find the cities with the minimum values
minima = df_pit_clean.describe().loc['min']
for i, value in minima.items():
    print(df_pit_clean[df_pit_clean[i]==value]['CoC Name']);
    
# CoCs with minima: "Garrett County" "Franklin, Essex Counties"

166    Garrett County CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
269    Franklin, Essex Counties CoC
Name: CoC Name, dtype: object
269    Franklin, Essex Counties CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
166    Garrett County CoC
Name: CoC Name, dtype: object
269    Franklin, Essex Counties CoC
Name: CoC Name, dtype: object
269    Franklin, Essex Counties CoC
Name: CoC Name, dtype: object


# Web Scraping for City Names from CoC Numbers

In [25]:
# making dataframe of cities associated with CoCs: df_coc
# scrape from https://www.hudexchange.info/grantees/find-a-grantee using BeautifulSoup


# if needed: pip install requests or conda install requests
import requests

requests.__path__

url = 'https://www.hudexchange.info/grantees/ak-500/'
# general form of url: "https://www.hudexchange.info/grantees/{}/".format(coc) to insert CoC ID from for loop

response = requests.get(url)

In [26]:
response.status_code # status code = 200 => OK

200

In [27]:
print(response.text)

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<link href='https://fonts.googleapis.com/css?family=Open+Sans:400,700,300,600' rel='stylesheet' type='text/css'>
<link href="https://fonts.googleapis.com/css?family=Poppins:600" rel="stylesheet" type='text/css'>
<meta name="description" content="" />
<meta name="keywords" content="" />
<meta name="viewport" content="width = device-width, initial-scale = 1.0" />
<meta name="HUDX_ct" content="Resources">
<title>AK-500 - HUD Exchange</title>
<link rel="icon" href="/onecpd/includes/themes/hudexchange/images/favicon.ico" type="image/x-icon" />
<link rel="shortcut icon" href="/onecpd/includes/themes/hudexchange/images/favicon.ico" type="image/x-icon" />
<!-- jQuery-UI -->
<link rel="stylesheet" type="text/css" href="/onecpd/includes/themes/hudexchange/css/ui/jquery-ui-1.12.1.min.css" />
<!-- special animation library -->
<link 

In [28]:
page_test = response.text

In [29]:
# if needed: pip install beautifulsoup4 lxml or conda install beautifulsoup4 lxml
from bs4 import BeautifulSoup

In [30]:
soup = BeautifulSoup(page_test, "lxml")
soup

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,700,300,600" rel="stylesheet" type="text/css"/>
<link href="https://fonts.googleapis.com/css?family=Poppins:600" rel="stylesheet" type="text/css"/>
<meta content="" name="description"/>
<meta content="" name="keywords"/>
<meta content="width = device-width, initial-scale = 1.0" name="viewport"/>
<meta content="Resources" name="HUDX_ct"/>
<title>AK-500 - HUD Exchange</title>
<link href="/onecpd/includes/themes/hudexchange/images/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/onecpd/includes/themes/hudexchange/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<!-- jQuery-UI -->
<link href="/onecpd/includes/themes/hudexchange/css/ui/jquery-ui-1.12.1.min.css" rel="stylesheet" type="text/css"/>
<!-- special animation library -->
<link hre

In [31]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-us">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,700,300,600" rel="stylesheet" type="text/css"/>
  <link href="https://fonts.googleapis.com/css?family=Poppins:600" rel="stylesheet" type="text/css"/>
  <meta content="" name="description"/>
  <meta content="" name="keywords"/>
  <meta content="width = device-width, initial-scale = 1.0" name="viewport"/>
  <meta content="Resources" name="HUDX_ct"/>
  <title>
   AK-500 - HUD Exchange
  </title>
  <link href="/onecpd/includes/themes/hudexchange/images/favicon.ico" rel="icon" type="image/x-icon"/>
  <link href="/onecpd/includes/themes/hudexchange/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <!-- jQuery-UI -->
  <link href="/onecpd/includes/themes/hudexchange/css/ui/jquery-ui-1.12.1.min.css" rel="stylesheet" type="text/css"/>
  <!-- spe

In [32]:
import time

In [33]:
import re

In [34]:
# test web scraping for loop
city_state_zip_regex = r'^(.+?),\s\w\w\s\b\d{5}\b'
for element in soup.find(class_='col-md-3 mb-1'):
    # if str of element matches the regex of some biz. regex = ^(.+?),\s\w\w\s\b\d{5}\b
    if re.search(city_state_zip_regex, str(element)):
        # store element somewhere
        print(element + "This one!")
    print(element)
    time.sleep(1)



<h4>Collaborative Applicant</h4>
Jasmine Khan
<br/>
Coalition Director
<br/>
The Anchorage Coalition to End Homelessness
<br/>
3201 C Street, Suite 110
<br/>
Anchorage, AK 99503This one!
Anchorage, AK 99503


KeyboardInterrupt: 

In [ ]:
# test web scraping for another url
url_test = 'https://www.hudexchange.info/grantees/AL-505/'
response = requests.get(url_test)
response.status_code # status code = 200 => OK

In [ ]:
page_test = response.text
soup = BeautifulSoup(page_test, "lxml")


In [ ]:
city_state_zip_regex = r'^(.+?),\s\w\w\s\b\d{5}\b'
for element in soup.find(class_='col-md-3 mb-1'):
    # if str of element matches the regex of some biz. regex = ^(.+?),\s\w\w\s\b\d{5}\b
    if re.search(city_state_zip_regex, str(element)):
        # store element somewhere
        print(element + "This one!")
    print(element)
    time.sleep(1)

In [ ]:
# test web scraping for another url
url_test = 'https://www.hudexchange.info/grantees/MO-501/'
response = requests.get(url_test)
response.status_code # status code = 200 => OK

In [ ]:
page_test = response.text
soup = BeautifulSoup(page_test, "lxml")


In [ ]:
city_state_zip_regex = r'^(.+?),\s\w\w\s\b\d{5}\b'
for element in soup.find(class_='col-md-3 mb-1'):
    # if str of element matches the regex of some biz. regex = ^(.+?),\s\w\w\s\b\d{5}\b
    if re.search(city_state_zip_regex, str(element)):
        # store element somewhere
        print(element + "This one!")
    print(element)
    time.sleep(1)

In [ ]:
# make a list of elements in "CoC Number" column
coc_number = [num for num in df_pit_clean['CoC Number']]

In [ ]:
from collections import OrderedDict
# I need an orderedDict because my web scraped address values will need to be merged with my dataframe in the right order later.

In [ ]:
# test again: the first 5 CoCs
city_state_zip_regex = r'^(.+?),\s\w\w\s\b\d{5}\b'
for num in coc_number[:5]:
    city_of_coc = OrderedDict()
    url = f'https://www.hudexchange.info/grantees/{num}/'
    response = requests.get(url)
    if response.status_code != 200: # status code = 200 => OK
        print("Not good!")
        break
    soup = BeautifulSoup(response.text, "lxml")
    for element in soup.find(class_='col-md-3 mb-1'):
        # if str of element matches the regex of some biz. regex = ^(.+?),\s\w\w\s\b\d{5}\b
        if re.search(city_state_zip_regex, str(element)):
            # store element in an ordered dictionary with key=num : value=element
            city_of_coc.update({num : element})
            print("YAY!")
        time.sleep(.1)
    


In [35]:
def find_city_state_zip(start_index=0, end_index=None):
    '''
    Outputs a list of dicts where key=CoC number and value=City, State, and Zip from url.
    '''
    # make a list out of elements in "CoC Number" column
    output = []
    coc_number = [num for num in df_pit_clean['CoC Number']]
    city_state_zip_regex = re.compile(r'^(.+?),\s\w\w\s\b\d{5}\b')
    # this (outer) loop finds the url and makes sure it's working,
    # then soupifies the requested url.
    for index, num in enumerate(coc_number[start_index:end_index]):
        # iterative report of passing urls
        if index % 20 == 0:
            print(f"Complete through index: {index+start_index}")
        
        url = f'https://www.hudexchange.info/grantees/{num}/'
        response = requests.get(url)
        if response.status_code != 200: # status code = 200 => OK
            print(f"Not good! {num} at index {index+start_index} does not work!")
            break
        soup = BeautifulSoup(response.text, "lxml")
        # this (nested) loop copies the correct line from the request to the output
        for element in soup.find(class_='col-md-3 mb-1'):
            # if str of element matches the regex
            if re.search(city_state_zip_regex, str(element)):
                # store element in an ordered dictionary with key=num : value=element
                output.append({"CoC Number": num, "city_state_zip": str(element)})
    return output


In [ ]:
# >>> d = {'col1': [1, 2], 'col2': [3, 4]}
# >>> df = pd.DataFrame(data=d)
# >>> df
#    col1  col2
# 0     1     3
# 1     2     4

In [36]:
city_state_zip = find_city_state_zip()

Complete through index: 0
Complete through index: 20
Complete through index: 40
Complete through index: 60
Complete through index: 80
Complete through index: 100
Complete through index: 120
Complete through index: 140
Complete through index: 160
Complete through index: 180
Complete through index: 200
Complete through index: 220
Complete through index: 240
Complete through index: 260
Complete through index: 280
Complete through index: 300
Complete through index: 320
Complete through index: 340
Complete through index: 360
Complete through index: 380


# Cleaning and Formatting City/State/Zip Data

In [37]:
city_state_zip

[{'CoC Number': 'AK-500', 'city_state_zip': 'Anchorage, AK 99503'},
 {'CoC Number': 'AK-501', 'city_state_zip': 'Anchorage, AK 99510'},
 {'CoC Number': 'AL-500', 'city_state_zip': 'Birmingham, AL 35233'},
 {'CoC Number': 'AL-501', 'city_state_zip': 'Mobile, AL 36609'},
 {'CoC Number': 'AL-502', 'city_state_zip': 'Florence, AL 35630'},
 {'CoC Number': 'AL-503', 'city_state_zip': 'Huntsville, AL 35816'},
 {'CoC Number': 'AL-504', 'city_state_zip': 'Montgomery, AL 36109'},
 {'CoC Number': 'AL-505', 'city_state_zip': 'Birmingham, AL 35203-5302'},
 {'CoC Number': 'AL-506', 'city_state_zip': 'Tuscaloosa, AL 35401'},
 {'CoC Number': 'AL-507', 'city_state_zip': 'Montgomery, AL 36101'},
 {'CoC Number': 'AR-500', 'city_state_zip': 'Little Rock, AR 72201'},
 {'CoC Number': 'AR-501', 'city_state_zip': 'Springdale, AR 72762'},
 {'CoC Number': 'AR-505', 'city_state_zip': 'Pine Bluff , AR 71601'},
 {'CoC Number': 'AZ-500', 'city_state_zip': 'Phoenix, AZ 85007'},
 {'CoC Number': 'AZ-501', 'city_state_

In [38]:
city_state_zip_df = pd.DataFrame(city_state_zip).copy()
city_state_zip_df

,CoC Number,city_state_zip
0,AK-500,"Anchorage, AK 99503"
1,AK-501,"Anchorage, AK 99510"
2,AL-500,"Birmingham, AL 35233"
3,AL-501,"Mobile, AL 36609"
4,AL-502,"Florence, AL 35630"
...,...,...
379,WV-500,"Wheeling, WV 26003"
380,WV-501,"Huntington, WV 25701"
381,WV-503,"Charleston, WV 25301"
382,WV-508,"Bridgeport, WV 26330"


In [ ]:
# trying to look at AZ cities to see how they're formatted
# city_state_zip_df[city_state_zip_df['CoC Number']]

In [39]:
city_state_zip_df.describe()

,CoC Number,city_state_zip
count,384,384
unique,384,373
top,MD-511,"Albany, NY 12210"
freq,1,8


In [40]:
# for some reason there are multiple CoCs for some cities. Explore this.
count_values = city_state_zip_df["city_state_zip"].value_counts()
type(count_values)

pandas.core.series.Series

In [41]:
# s = s[s != 1]
multi_values = count_values[count_values != 1]

In [45]:
multi_values

Albany, NY 12210          8
St. Paul, MN 55114        2
Harrisburg, PA 17120      2
Denver, CO 80205          2
Indianapolis, IN 46204    2
Name: city_state_zip, dtype: int64

In [47]:
multi_values = pd.DataFrame(multi_values)
multi_values

,city_state_zip
"Albany, NY 12210",8
"St. Paul, MN 55114",2
"Harrisburg, PA 17120",2
"Denver, CO 80205",2
"Indianapolis, IN 46204",2


In [48]:
multi_values.reset_index(inplace=True)
multi_values

,index,city_state_zip
0,"Albany, NY 12210",8
1,"St. Paul, MN 55114",2
2,"Harrisburg, PA 17120",2
3,"Denver, CO 80205",2
4,"Indianapolis, IN 46204",2


In [49]:
multi_values.rename(columns={"index":"city_state_zip", "city_state_zip":"multi_coc"},inplace=True)
multi_values

,city_state_zip,multi_coc
0,"Albany, NY 12210",8
1,"St. Paul, MN 55114",2
2,"Harrisburg, PA 17120",2
3,"Denver, CO 80205",2
4,"Indianapolis, IN 46204",2


In [77]:
# trying to view the rows of city_state_zip_df that are also in multi_values
# something like city_state_zip_df[city_state_zip_df["city_state_zip"].value_counts()>=2] (doesn't work)

# df = pd.merge(df1, df2, on=['User','Movie'], how='left', indicator='Exist')
# df.drop('Rating', inplace=True, axis=1)
# df['Exist'] = np.where(df.Exist == 'both', True, False)

# df_pit_all = pd.merge(df_pit_all, df_pit_2016[['CoC Number','Overall Homeless, 2016']], on='CoC Number', how='outer')

city_state_zip_df_multi = pd.merge(city_state_zip_df, multi_values[['city_state_zip', 'multi_coc']], on='city_state_zip', how='left')


In [78]:
city_state_zip_df_multi

,CoC Number,city_state_zip,multi_coc
0,AK-500,"Anchorage, AK 99503",NaN
1,AK-501,"Anchorage, AK 99510",NaN
2,AL-500,"Birmingham, AL 35233",NaN
3,AL-501,"Mobile, AL 36609",NaN
4,AL-502,"Florence, AL 35630",NaN
...,...,...,...
379,WV-500,"Wheeling, WV 26003",NaN
380,WV-501,"Huntington, WV 25701",NaN
381,WV-503,"Charleston, WV 25301",NaN
382,WV-508,"Bridgeport, WV 26330",NaN


In [85]:
city_state_zip_df_multi.fillna(value=0,inplace=True)
city_state_zip_df_multi

,CoC Number,city_state_zip,multi_coc
0,AK-500,"Anchorage, AK 99503",0.0
1,AK-501,"Anchorage, AK 99510",0.0
2,AL-500,"Birmingham, AL 35233",0.0
3,AL-501,"Mobile, AL 36609",0.0
4,AL-502,"Florence, AL 35630",0.0
...,...,...,...
379,WV-500,"Wheeling, WV 26003",0.0
380,WV-501,"Huntington, WV 25701",0.0
381,WV-503,"Charleston, WV 25301",0.0
382,WV-508,"Bridgeport, WV 26330",0.0


In [86]:
city_state_zip_df_multi.iloc[230:260,]

,CoC Number,city_state_zip,multi_coc
230,NE-501,"Omaha, NE 68182",0.0
231,NE-502,"Lincoln, NE 68508",0.0
232,NH-500,"Concord, NH 03301",0.0
233,NH-501,"Manchester, NH 03101",0.0
234,NH-502,"Nashua, NH 03062",0.0
235,NJ-500,"Atlantic City, NJ 08401",0.0
236,NJ-501,"Hackensack, NJ 07601",0.0
237,NJ-502,"Mount Holly, NJ 08060",0.0
238,NJ-503,"Pennsauken, NJ 08109",0.0
239,NJ-504,"East Orange, NJ 07018",0.0


In [87]:
# cities that correspond to multiple CoC Numbers
city_state_zip_df_multi[city_state_zip_df_multi['multi_coc'] != 0]

,CoC Number,city_state_zip,multi_coc
56,CO-500,"Denver, CO 80205",2.0
57,CO-503,"Denver, CO 80205",2.0
125,IN-502,"Indianapolis, IN 46204",2.0
126,IN-503,"Indianapolis, IN 46204",2.0
197,MN-503,"St. Paul, MN 55114",2.0
198,MN-504,"St. Paul, MN 55114",2.0
256,NY-501,"Albany, NY 12210",8.0
257,NY-503,"Albany, NY 12210",8.0
260,NY-507,"Albany, NY 12210",8.0
263,NY-512,"Albany, NY 12210",8.0


# Import and Clean City Population Data

In [95]:
column_labels='SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018'
columns_list = column_labels.split(",")
columns_list

['SUMLEV',
 'STATE',
 'COUNTY',
 'PLACE',
 'COUSUB',
 'CONCIT',
 'PRIMGEO_FLAG',
 'FUNCSTAT',
 'NAME',
 'STNAME',
 'CENSUS2010POP',
 'ESTIMATESBASE2010',
 'POPESTIMATE2010',
 'POPESTIMATE2011',
 'POPESTIMATE2012',
 'POPESTIMATE2013',
 'POPESTIMATE2014',
 'POPESTIMATE2015',
 'POPESTIMATE2016',
 'POPESTIMATE2017',
 'POPESTIMATE2018']

## import

In [101]:
# csv info from https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2018/sub-est2018.pdf
# import csv
csv_url = r'https://www2.census.gov/programs-surveys/popest/datasets/2010-2018/cities/totals/sub-est2018_all.csv'
csv_file = 'annual_population_estimates_all.csv' # saved csv file with utf-8 encoding using Sublime Text.

city_pop_data = pd.read_csv(csv_file)
city_pop_data

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,...,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
0,40,1,0,0,0,0,0,A,Alabama,Alabama,...,4780138,4785448,4798834,4815564,4830460,4842481,4853160,4864745,4875120,4887871
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,...,2688,2684,2677,2629,2612,2595,2588,2573,2566,2563
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,...,4516,4510,4496,4474,4453,4432,4406,4373,4342,4325
3,162,1,0,484,0,0,0,A,Addison town,Alabama,...,754,751,751,744,743,740,735,734,728,724
4,162,1,0,676,0,0,0,A,Akron town,Alabama,...,356,355,345,345,341,339,337,332,332,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81431,157,56,43,99990,0,0,1,F,Balance of Washakie County,Wyoming,...,2782,2782,2755,2733,2736,2689,2690,2651,2606,2556
81432,50,56,45,0,0,0,0,A,Weston County,Wyoming,...,7208,7199,7142,7077,7137,7138,7197,7213,6986,6967
81433,157,56,45,56215,0,0,1,A,Newcastle city,Wyoming,...,3533,3531,3506,3473,3481,3481,3502,3521,3412,3402
81434,157,56,45,79125,0,0,1,A,Upton town,Wyoming,...,1100,1098,1089,1083,1090,1097,1105,1107,1067,1064


In [104]:
# data from previous years: https://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/cities/sub-est00int.csv
prev_csv_url = r'https://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/cities/sub-est00int.csv'
prev_csv = 'sub-est00int.csv'
city_pop_data_prev = pd.read_csv(prev_csv, encoding='utf-8')
city_pop_data_prev

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,NAME,STNAME,ESTIMATESBASE2000,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP,POPESTIMATE2010
0,40,1,0,0,0,Alabama,Alabama,4447207,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938,4779736,4785298
1,162,1,0,124,0,Abbeville city,Alabama,2989,2985,2941,2909,2882,2857,2820,2807,2784,2742,2714,2688,2689
2,162,1,0,460,0,Adamsville city,Alabama,5033,5021,4960,4894,4841,4784,4728,4687,4633,4594,4558,4522,4523
3,162,1,0,484,0,Addison town,Alabama,698,701,701,708,714,722,729,741,750,752,759,758,755
4,162,1,0,676,0,Akron town,Alabama,488,485,473,454,442,426,416,404,395,384,369,356,355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81620,157,56,43,99990,0,Balance of Washakie County,Wyoming,2550,2539,2500,2490,2502,2514,2548,2550,2626,2660,2739,2786,2793
81621,50,56,45,0,0,Weston County,Wyoming,6643,6627,6487,6578,6610,6646,6594,6717,7033,7133,7266,7208,7182
81622,157,56,45,56215,0,Newcastle city,Wyoming,3313,3298,3224,3259,3268,3283,3246,3297,3471,3506,3565,3532,3520
81623,157,56,45,79125,0,Upton town,Wyoming,881,884,878,906,924,942,949,982,1034,1065,1098,1100,1096


In [ ]:
# use census data where possible, otherwise use population estimates.

In [105]:
city_pop_data.columns, city_pop_data_prev.columns

(Index(['SUMLEV', 'STATE', 'COUNTY', 'PLACE', 'COUSUB', 'CONCIT',
        'PRIMGEO_FLAG', 'FUNCSTAT', 'NAME', 'STNAME', 'CENSUS2010POP',
        'ESTIMATESBASE2010', 'POPESTIMATE2010', 'POPESTIMATE2011',
        'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
        'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
        'POPESTIMATE2018'],
       dtype='object'),
 Index(['SUMLEV', 'STATE', 'COUNTY', 'PLACE', 'COUSUB', 'NAME', 'STNAME',
        'ESTIMATESBASE2000', 'POPESTIMATE2000', 'POPESTIMATE2001',
        'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
        'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
        'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP',
        'POPESTIMATE2010'],
       dtype='object'))

In [107]:
pop_df = pd.merge(city_pop_data, 
                  city_pop_data_prev, 
                  on=['NAME', 'STNAME'], 
                  how='left')

In [108]:
#columns to keep: ['SUMLEV', 'STATE', 'COUNTY', 'PLACE', 'COUSUB', 'NAME', 'STNAME', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP']
pop_df

,SUMLEV_x,STATE_x,COUNTY_x,PLACE_x,COUSUB_x,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,...,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP_y,POPESTIMATE2010_y
0,40,1,0,0,0,0,0,A,Alabama,Alabama,...,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0,4779736.0,4785298.0
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,...,2909.0,2882.0,2857.0,2820.0,2807.0,2784.0,2742.0,2714.0,2688.0,2689.0
2,162,1,0,124,0,0,0,A,Abbeville city,Alabama,...,2909.0,2882.0,2857.0,2820.0,2807.0,2784.0,2742.0,2714.0,2688.0,2689.0
3,162,1,0,460,0,0,0,A,Adamsville city,Alabama,...,4894.0,4841.0,4784.0,4728.0,4687.0,4633.0,4594.0,4558.0,4522.0,4523.0
4,162,1,0,460,0,0,0,A,Adamsville city,Alabama,...,4894.0,4841.0,4784.0,4728.0,4687.0,4633.0,4594.0,4558.0,4522.0,4523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223912,157,56,45,56215,0,0,1,A,Newcastle city,Wyoming,...,3259.0,3268.0,3283.0,3246.0,3297.0,3471.0,3506.0,3565.0,3532.0,3520.0
223913,157,56,45,56215,0,0,1,A,Newcastle city,Wyoming,...,3259.0,3268.0,3283.0,3246.0,3297.0,3471.0,3506.0,3565.0,3532.0,3520.0
223914,157,56,45,79125,0,0,1,A,Upton town,Wyoming,...,906.0,924.0,942.0,949.0,982.0,1034.0,1065.0,1098.0,1100.0,1096.0
223915,157,56,45,79125,0,0,1,A,Upton town,Wyoming,...,906.0,924.0,942.0,949.0,982.0,1034.0,1065.0,1098.0,1100.0,1096.0


## clean

In [109]:
pop_2007_2018=pop_df.copy()

In [111]:
pop_2007_2018.columns

Index(['SUMLEV_x', 'STATE_x', 'COUNTY_x', 'PLACE_x', 'COUSUB_x', 'CONCIT',
       'PRIMGEO_FLAG', 'FUNCSTAT', 'NAME', 'STNAME', 'CENSUS2010POP_x',
       'ESTIMATESBASE2010', 'POPESTIMATE2010_x', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'SUMLEV_y', 'STATE_y', 'COUNTY_y', 'PLACE_y',
       'COUSUB_y', 'ESTIMATESBASE2000', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP_y',
       'POPESTIMATE2010_y'],
      dtype='object')

In [114]:
# keep only the columns I need
# df1 = df[['a','d']]
pop_clean= pop_2007_2018[['NAME', 'STNAME', 'STATE_x', 'SUMLEV_x', 'COUNTY_x', 'PLACE_x', 'COUSUB_x', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP_x',
       'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018']].copy()

In [115]:
pop_clean

,NAME,STNAME,STATE_x,SUMLEV_x,COUNTY_x,PLACE_x,COUSUB_x,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP_x,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
0,Alabama,Alabama,1,40,0,0,0,4672840.0,4718206.0,4757938.0,4779736,4798834,4815564,4830460,4842481,4853160,4864745,4875120,4887871
1,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
2,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
3,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
4,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223912,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223913,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223914,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064
223915,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064


In [131]:
# rename columns for ease of use
# gapminder.rename(columns={'pop':'population',
#                           'lifeExp':'life_exp',
#                           'gdpPercap':'gdp_per_cap'}, 
#                  inplace=True)
pop_clean.rename(columns={'NAME':'area_name', 'STNAME':'state_name', 'STATE_x':'state_fips', 
                          'SUMLEV_x':'summary_level', 'COUNTY_x':'county_fips', 'PLACE_x':'place_fips', 
                          'COUSUB_x':'county_sub_fips', 'POPESTIMATE2007':'pop_2007','POPESTIMATE2008':'pop_2008',
                          'POPESTIMATE2009':'pop_2009', 'CENSUS2010POP_y':'pop_2010','POPESTIMATE2011':'pop_2011',
                          'POPESTIMATE2012':'pop_2012', 'POPESTIMATE2013':'pop_2013','POPESTIMATE2014':'pop_2014',
                          'POPESTIMATE2015':'pop_2015', 'POPESTIMATE2016':'pop_2016','POPESTIMATE2017':'pop_2017',
                          'POPESTIMATE2018':'pop_2018'}, inplace=True)
pop_clean

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
0,Alabama,Alabama,1,40,0,0,0,4672840.0,4718206.0,4757938.0,4779736,4798834,4815564,4830460,4842481,4853160,4864745,4875120,4887871
1,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
2,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
3,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
4,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223912,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223913,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223914,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064
223915,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064


In [132]:
pop_clean.dtypes

area_name           object
state_name          object
state_fips           int64
summary_level        int64
county_fips          int64
place_fips           int64
county_sub_fips      int64
pop_2007           float64
pop_2008           float64
pop_2009           float64
pop_2010            object
pop_2011             int64
pop_2012             int64
pop_2013             int64
pop_2014             int64
pop_2015             int64
pop_2016             int64
pop_2017             int64
pop_2018             int64
dtype: object

In [133]:
# pop_clean['pop_2010']=pd.to_numeric(pop_clean['pop_2010'])
pop_clean['pop_2010'].value_counts()
# why is population sometimes A?

0         1925
A          469
2          332
61         322
86         295
          ... 
57618        1
345491       1
164981       1
11629        1
6940         1
Name: pop_2010, Length: 15726, dtype: int64

In [134]:
pop_clean[pop_clean['pop_2010']=="A"]

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
2146,Kachemak city,Alaska,2,162,0,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2147,Kachemak city,Alaska,2,162,0,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2437,Kachemak city,Alaska,2,157,122,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2438,Kachemak city,Alaska,2,157,122,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2599,Kake city,Alaska,2,157,198,36770,0,593.0,572.0,558.0,A,572,577,576,576,579,578,572,563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219029,Ontario village,Wisconsin,55,61,81,0,60075,534.0,540.0,550.0,A,0,0,0,0,0,0,0,0
219034,Rockland village,Wisconsin,55,61,81,0,68900,598.0,596.0,595.0,A,0,0,0,0,0,0,0,0
219035,Rockland village,Wisconsin,55,61,81,0,68900,598.0,596.0,595.0,A,0,0,0,0,0,0,0,0
219036,Rockland village,Wisconsin,55,61,81,0,68900,598.0,596.0,595.0,A,0,0,0,0,0,0,0,0


In [135]:
pop_clean[pop_clean['area_name'] == 'Kachemak city']

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
2146,Kachemak city,Alaska,2,162,0,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2147,Kachemak city,Alaska,2,162,0,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2437,Kachemak city,Alaska,2,157,122,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490
2438,Kachemak city,Alaska,2,157,122,36540,0,455.0,457.0,467.0,A,482,486,487,489,491,493,492,490


In [136]:
pop_clean.dropna(inplace=True)

In [137]:
pop_clean[pop_clean['pop_2010']=="A"]

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
0,Alabama,Alabama,1,40,0,0,0,4672840.0,4718206.0,4757938.0,4779736,4798834,4815564,4830460,4842481,4853160,4864745,4875120,4887871
1,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
2,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
3,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
4,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223912,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223913,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223914,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064
223915,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064


In [138]:
pop_clean['area_name'].value_counts()

Washington township                      4881
Jackson township                         4066
Union township                           2741
Jefferson township                       1527
Liberty township                         1428
                                         ... 
Balance of Shackelford County               1
Lake Mary township                          1
Lower Chichester township                   1
Balance of Silver Lake township             1
Balance of Valdez-Cordova Census Area       1
Name: area_name, Length: 31606, dtype: int64

In [140]:
# remove population counts for states: where summary_level = 40
# because MVP will measure at city/CoC level
sumlev_not_40 = pop_clean.summary_level != 40
sumlev_not_40, sumlev_not_40.shape

(0         False
 1          True
 2          True
 3          True
 4          True
           ...  
 223912     True
 223913     True
 223914     True
 223915     True
 223916     True
 Name: summary_level, Length: 223422, dtype: bool, (223422,))

In [141]:
pop_clean = pop_clean[sumlev_not_40]
pop_clean, pop_clean.shape # removes 52 rows, as expected, removing 52 state-level areas

(                       area_name state_name  state_fips  summary_level  \
 1                 Abbeville city    Alabama           1            162   
 2                 Abbeville city    Alabama           1            162   
 3                Adamsville city    Alabama           1            162   
 4                Adamsville city    Alabama           1            162   
 5                   Addison town    Alabama           1            162   
 ...                          ...        ...         ...            ...   
 223912            Newcastle city    Wyoming          56            157   
 223913            Newcastle city    Wyoming          56            157   
 223914                Upton town    Wyoming          56            157   
 223915                Upton town    Wyoming          56            157   
 223916  Balance of Weston County    Wyoming          56            157   
 
         county_fips  place_fips  county_sub_fips  pop_2007  pop_2008  \
 1                 0     

In [143]:
# look at city-level areas
sumlev_city_level_code = [170,172]
pop_clean[pop_clean.summary_level.isin(sumlev_city_level_code)] # gives only 233 rows, apparently many duplicates

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
8611,Milford city (balance),Connecticut,9,172,0,47515,0,51203.0,51221.0,51242.0,51271,51383,51560,51668,51988,52146,52550,52826,53120
8612,Milford city (balance),Connecticut,9,172,0,47515,0,51203.0,51221.0,51242.0,51271,51383,51560,51668,51988,52146,52550,52826,53120
8613,Milford city (balance),Connecticut,9,172,0,47515,0,51203.0,51221.0,51242.0,51271,51383,51560,51668,51988,52146,52550,52826,53120
8614,Woodmont borough,Connecticut,9,172,0,88050,0,1538.0,1520.0,1501.0,1488,1490,1496,1499,1507,1513,1527,1532,1541
8615,Woodmont borough,Connecticut,9,172,0,88050,0,1538.0,1520.0,1501.0,1488,1490,1496,1499,1507,1513,1527,1532,1541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199107,Nashville-Davidson metropolitan government (ba...,Tennessee,47,172,0,52006,0,580162.0,587567.0,595682.0,601222,612422,625601,636267,645820,656572,663339,665396,669053
199108,Nashville-Davidson metropolitan government (ba...,Tennessee,47,172,0,52006,0,580162.0,587567.0,595682.0,601222,612422,625601,636267,645820,656572,663339,665396,669053
199109,Oak Hill city,Tennessee,47,172,0,54780,0,4466.0,4489.0,4517.0,4529,4573,4643,4686,4701,4693,4676,4574,4553
199110,Oak Hill city,Tennessee,47,172,0,54780,0,4466.0,4489.0,4517.0,4529,4573,4643,4686,4701,4693,4676,4574,4553


In [145]:
pop_clean = pop_clean.drop_duplicates()

In [146]:
pop_clean[pop_clean.summary_level.isin(sumlev_city_level_code)]

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
8611,Milford city (balance),Connecticut,9,172,0,47515,0,51203.0,51221.0,51242.0,51271,51383,51560,51668,51988,52146,52550,52826,53120
8614,Woodmont borough,Connecticut,9,172,0,88050,0,1538.0,1520.0,1501.0,1488,1490,1496,1499,1507,1513,1527,1532,1541
12034,Athens-Clarke County unified government (balance),Georgia,13,172,0,3440,0,112713.0,114485.0,115566.0,115452,116971,118643,119629,119131,122252,123559,125462,125964
12036,Augusta-Richmond County consolidated governmen...,Georgia,13,172,0,4204,0,191898.0,193797.0,193825.0,195844,195643,196834,196479,196473,196663,197146,196828,196939
12038,Blythe city,Georgia,13,172,0,9040,0,706.0,712.0,716.0,694,689,693,689,687,684,680,678,674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199104,Forest Hills city,Tennessee,47,172,0,27020,0,4739.0,4765.0,4798.0,4812,4861,4941,4989,5040,5050,5007,4914,4866
199106,Goodlettsville city,Tennessee,47,172,0,29920,0,15196.0,15468.0,15743.0,10319,10460,10659,10930,11012,10973,10845,10642,10663
199107,Nashville-Davidson metropolitan government (ba...,Tennessee,47,172,0,52006,0,580162.0,587567.0,595682.0,601222,612422,625601,636267,645820,656572,663339,665396,669053
199109,Oak Hill city,Tennessee,47,172,0,54780,0,4466.0,4489.0,4517.0,4529,4573,4643,4686,4701,4693,4676,4574,4553


In [147]:
# look at county-level areas
sumlev_county_level_code = [157,162]
pop_clean[pop_clean.summary_level.isin(sumlev_county_level_code)]

,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018
1,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563
3,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325
5,Addison town,Alabama,1,162,0,484,0,750.0,752.0,759.0,758,751,744,743,740,735,734,728,724
7,Akron town,Alabama,1,162,0,676,0,395.0,384.0,369.0,356,345,345,341,339,337,332,332,330
9,Alabaster city,Alabama,1,162,0,820,0,29020.0,29630.0,30107.0,30352,31346,31653,31946,32149,32739,32984,33223,33340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223908,Worland city,Wyoming,56,157,43,84925,0,5268.0,5301.0,5420.0,5487,5435,5419,5420,5330,5334,5263,5173,5075
223910,Balance of Washakie County,Wyoming,56,157,43,99990,0,2626.0,2660.0,2739.0,2786,2755,2733,2736,2689,2690,2651,2606,2556
223912,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402
223914,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064


In [149]:
pop_clean['state_fips']

1          1
3          1
5          1
7          1
9          1
          ..
223910    56
223911    56
223912    56
223914    56
223916    56
Name: state_fips, Length: 130453, dtype: int64

## add State Abbreviation column

In [156]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}
# from http://code.activestate.com/recipes/577775-state-fips-codes-dict/

In [163]:
state_codes_ints = {int(val):key for key, val in state_codes.items()}

In [164]:
state_codes_ints

{53: 'WA',
 10: 'DE',
 11: 'DC',
 55: 'WI',
 54: 'WV',
 15: 'HI',
 12: 'FL',
 56: 'WY',
 72: 'PR',
 34: 'NJ',
 35: 'NM',
 48: 'TX',
 22: 'LA',
 37: 'NC',
 38: 'ND',
 31: 'NE',
 47: 'TN',
 36: 'NY',
 42: 'PA',
 2: 'AK',
 32: 'NV',
 33: 'NH',
 51: 'VA',
 8: 'CO',
 6: 'CA',
 1: 'AL',
 5: 'AR',
 50: 'VT',
 17: 'IL',
 13: 'GA',
 18: 'IN',
 19: 'IA',
 25: 'MA',
 4: 'AZ',
 16: 'ID',
 9: 'CT',
 23: 'ME',
 24: 'MD',
 40: 'OK',
 39: 'OH',
 49: 'UT',
 29: 'MO',
 27: 'MN',
 26: 'MI',
 44: 'RI',
 20: 'KS',
 30: 'MT',
 28: 'MS',
 45: 'SC',
 21: 'KY',
 41: 'OR',
 46: 'SD'}

In [165]:
pop_clean['state_abbrev']=pop_clean['state_fips'].map(state_codes_ints)
pop_clean

/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,area_name,state_name,state_fips,summary_level,county_fips,place_fips,county_sub_fips,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018,state_abbrev
1,Abbeville city,Alabama,1,162,0,124,0,2784.0,2742.0,2714.0,2688,2677,2629,2612,2595,2588,2573,2566,2563,AL
3,Adamsville city,Alabama,1,162,0,460,0,4633.0,4594.0,4558.0,4522,4496,4474,4453,4432,4406,4373,4342,4325,AL
5,Addison town,Alabama,1,162,0,484,0,750.0,752.0,759.0,758,751,744,743,740,735,734,728,724,AL
7,Akron town,Alabama,1,162,0,676,0,395.0,384.0,369.0,356,345,345,341,339,337,332,332,330,AL
9,Alabaster city,Alabama,1,162,0,820,0,29020.0,29630.0,30107.0,30352,31346,31653,31946,32149,32739,32984,33223,33340,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223910,Balance of Washakie County,Wyoming,56,157,43,99990,0,2626.0,2660.0,2739.0,2786,2755,2733,2736,2689,2690,2651,2606,2556,WY
223911,Weston County,Wyoming,56,50,45,0,0,7033.0,7133.0,7266.0,7208,7142,7077,7137,7138,7197,7213,6986,6967,WY
223912,Newcastle city,Wyoming,56,157,45,56215,0,3471.0,3506.0,3565.0,3532,3506,3473,3481,3481,3502,3521,3412,3402,WY
223914,Upton town,Wyoming,56,157,45,79125,0,1034.0,1065.0,1098.0,1100,1089,1083,1090,1097,1105,1107,1067,1064,WY


# Import and Clean City Land Area

In [168]:
land_area = pd.read_csv('land_area.csv')

In [169]:
land_area # what a mess!

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402
0,0400000US01,1,Alabama,0400000US01,1,Alabama,Alabama,4779736,2171853,52420.07,1774.74,50645.33,94.4,42.9
1,0400000US01,1,Alabama,1600000US0100100,100100,"Alabama - PLACE - Abanda CDP, Chambers County","Abanda CDP, Chambers County",192,79,3.01,0.01,3.00,64.0,26.4
2,0400000US01,1,Alabama,1600000US0100124,100124,"Alabama - PLACE - Abbeville city, Henry County","Abbeville city, Henry County",2688,1255,15.58,0.04,15.54,172.9,80.7
3,0400000US01,1,Alabama,1600000US0100460,100460,"Alabama - PLACE - Adamsville city, Jefferson C...","Adamsville city, Jefferson County",4522,1990,25.14,0.01,25.13,180.0,79.2
4,0400000US01,1,Alabama,1600000US0100484,100484,"Alabama - PLACE - Addison town, Winston County","Addison town, Winston County",758,351,3.80,0.03,3.77,201.3,93.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40833,0400000US72,72,Puerto Rico,1600000US7286014,7286014,"Puerto Rico - PLACE - Vieques zona urbana, Vie...","Vieques zona urbana, Vieques Municipio",1938,1173,0.88,0.30,0.58,3338.7,2020.8
40834,0400000US72,72,Puerto Rico,1600000US7286831,7286831,"Puerto Rico - PLACE - Villalba zona urbana, Vi...","Villalba zona urbana, Villalba Municipio",3384,1366,1.19,0.00,1.19,2832.4,1143.3
40835,0400000US72,72,Puerto Rico,1600000US7287863,7287863,"Puerto Rico - PLACE - Yabucoa zona urbana, Yab...","Yabucoa zona urbana, Yabucoa Municipio",6047,2795,1.15,0.00,1.15,5250.5,2426.9
40836,0400000US72,72,Puerto Rico,1600000US7288035,7288035,"Puerto Rico - PLACE - Yauco zona urbana, Yauco...","Yauco zona urbana, Yauco Municipio",17186,7512,2.88,0.00,2.87,5982.3,2614.9


In [172]:
land_area.head(50)

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402
0,0400000US01,1,Alabama,0400000US01,1,Alabama,Alabama,4779736,2171853,52420.07,1774.74,50645.33,94.4,42.9
1,0400000US01,1,Alabama,1600000US0100100,100100,"Alabama - PLACE - Abanda CDP, Chambers County","Abanda CDP, Chambers County",192,79,3.01,0.01,3.00,64.0,26.4
2,0400000US01,1,Alabama,1600000US0100124,100124,"Alabama - PLACE - Abbeville city, Henry County","Abbeville city, Henry County",2688,1255,15.58,0.04,15.54,172.9,80.7
3,0400000US01,1,Alabama,1600000US0100460,100460,"Alabama - PLACE - Adamsville city, Jefferson C...","Adamsville city, Jefferson County",4522,1990,25.14,0.01,25.13,180.0,79.2
4,0400000US01,1,Alabama,1600000US0100484,100484,"Alabama - PLACE - Addison town, Winston County","Addison town, Winston County",758,351,3.80,0.03,3.77,201.3,93.2
5,0400000US01,1,Alabama,1600000US0100676,100676,"Alabama - PLACE - Akron town, Hale County","Akron town, Hale County",356,205,0.69,0.01,0.69,518.5,298.6
6,0400000US01,1,Alabama,1600000US0100820,100820,"Alabama - PLACE - Alabaster city, Shelby County","Alabaster city, Shelby County",30352,11295,25.33,0.29,25.04,1212.0,451.0
7,0400000US01,1,Alabama,1600000US0100988,100988,"Alabama - PLACE - Albertville city, Marshall C...","Albertville city, Marshall County",21160,8128,26.66,0.10,26.56,796.8,306.1
8,0400000US01,1,Alabama,1600000US0101132,101132,"Alabama - PLACE - Alexander City city, Tallapo...","Alexander City city, Tallapoosa County",14875,6834,41.12,0.28,40.84,364.2,167.3
9,0400000US01,1,Alabama,1600000US0101180,101180,"Alabama - PLACE - Alexandria CDP, Calhoun County","Alexandria CDP, Calhoun County",3917,1599,11.13,0.01,11.11,352.4,143.9


In [ ]:
# from DEC_10_SF1_GCTPH1.ST10_metadata.csv
# GEO.id,Id
# GEO.id2,Id2
# GEO.display-label,Geography
# GCT_STUB.target-geo-id,Target Geo Id
# GCT_STUB.target-geo-id2,Target Geo Id2
# GCT_STUB.display-label,Geographic area,GCT_STUB.display-label
# HD01,Population
# HD02,Housing units
# SUBHD0301,Area in square miles - Total area
# SUBHD0302,Area in square miles - Water area
# SUBHD0303,Area in square miles - Land area
# SUBHD0401,Density per square mile of land area - Population
# SUBHD0402,Density per square mile of land area - Housing units

In [170]:
land_area.columns

Index(['GEO.id', 'GEO.id2', 'GEO.display-label', 'GCT_STUB.target-geo-id',
       'GCT_STUB.target-geo-id2', 'GCT_STUB.display-label',
       'GCT_STUB.display-label.1', 'HD01', 'HD02', 'SUBHD0301', 'SUBHD0302',
       'SUBHD0303', 'SUBHD0401', 'SUBHD0402'],
      dtype='object')

In [175]:
land_area.rename(columns = {'GEO.id2':'state_fips',
                            'GEO.display-label':'state_name',
                            'GCT_STUB.display-label.1':'city_county',
                            'SUBHD0303':'land_area', 
                            'HD02':'housing_units'}, inplace=True)

In [176]:
land_area.columns

Index(['GEO.id', 'state_fips', 'state_name', 'GCT_STUB.target-geo-id',
       'GCT_STUB.target-geo-id2', 'GCT_STUB.display-label', 'city_county',
       'HD01', 'housing_units', 'SUBHD0301', 'SUBHD0302', 'land_area',
       'SUBHD0401', 'SUBHD0402'],
      dtype='object')

In [178]:
land_area = land_area[['state_name','state_fips','city_county','land_area','housing_units']]

In [182]:
land_area.dtypes

state_name        object
state_fips         int64
city_county       object
land_area        float64
housing_units      int64
dtype: object

In [184]:
# add state_abbrev column like for pop_clean
land_area['state_abbrev']=land_area['state_fips'].map(state_codes_ints)
land_area

/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,state_name,state_fips,city_county,land_area,housing_units,state_abbrev
0,Alabama,1,Alabama,50645.33,2171853,AL
1,Alabama,1,"Abanda CDP, Chambers County",3.00,79,AL
2,Alabama,1,"Abbeville city, Henry County",15.54,1255,AL
3,Alabama,1,"Adamsville city, Jefferson County",25.13,1990,AL
4,Alabama,1,"Addison town, Winston County",3.77,351,AL
...,...,...,...,...,...,...
40833,Puerto Rico,72,"Vieques zona urbana, Vieques Municipio",0.58,1173,PR
40834,Puerto Rico,72,"Villalba zona urbana, Villalba Municipio",1.19,1366,PR
40835,Puerto Rico,72,"Yabucoa zona urbana, Yabucoa Municipio",1.15,2795,PR
40836,Puerto Rico,72,"Yauco zona urbana, Yauco Municipio",2.87,7512,PR


In [ ]:
# split city_county column into new columns like for city_state_zip_df


In [171]:
pop_clean.columns

Index(['area_name', 'state_name', 'state_fips', 'summary_level', 'county_fips',
       'place_fips', 'county_sub_fips', 'pop_2007', 'pop_2008', 'pop_2009',
       'pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014', 'pop_2015',
       'pop_2016', 'pop_2017', 'pop_2018', 'state_abbrev'],
      dtype='object')

# Save DataFrames as CSVs

In [ ]:
# df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv')
# r‘C:\Users\Ron\Desktop\export_dataframe.csv‘

In [89]:
df_pit_clean.to_csv('df_pit_clean.csv', index=False)

In [88]:
city_state_zip_df_multi.to_csv("city_state_zip_df_multi.csv", index=False)

In [167]:
pop_clean.to_csv("pop_clean.csv", index=False)

In [185]:
land_area.to_csv('land_area_df.csv', index=False)

# Appendix

In [ ]:
# exploring how this setup of loop works with a list; do not need to keep.
for i,j in enumerate(coc_number):
    print(i,"*", j)

In [ ]:
for i in range(4):
    print("YAY!")
    time.sleep(.5)

In [ ]:
df_pit_clean[df_pit_clean.index > 210].head(20)